# PyAVD - Quick Solver

In [1]:
from gpkit import Model, ureg
from gpkit.interactive.sankey import Sankey
from gpkit.constraints.bounded import Bounded
from Models import *
import gpkit.interactive
%matplotlib inline

In [2]:
# with gpkit.SignomialsEnabled():

#     AC = Aircraft()
#     # print(AC)

#     MISSION = Mission(AC)
#     # print(MISSION)

#     M = Model(AC.M_0 * AC.T0_W0, [Bounded(MISSION), Bounded(AC)])
#     print(M)

#     print(M.variables_byname('AR'))

#     sol = M.localsolve(verbosity=1)
#     print(sol.table())

#     print(M.program.results)

In [3]:

AC = Aircraft()
# print(AC)

MISSION = Mission(AC)
# print(MISSION)

# M = Model(AC.M_0 * AC.T0_W0, [Bounded(MISSION), Bounded(AC)])
M = Model(AC.M_0 * AC.T0_W0, [MISSION, AC])
print(M)

sol = M.solve(verbosity=1)
print(sol.table())

AttributeError: `M` is noted in Aircraft as unbounded, but is not an attribute of that model.

In [ ]:
# from gpkit.interactive.references import referencesplot
# referencesplot(M, openimmediately=True)

In [ ]:
# Sankey(sol, M)